In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('../data/data1.csv')
df = df.set_index('Date')
df

,S&P Market,S&P Communication Services,S&P Technology,S&P Industrial,S&P Materials,S&P Consumer Discretionary,S&P Financial,S&P Health Care,S&P Consumer Staples,S&P Utilities,S&P Real Estate,S&P Energy
Date,,,,,,,,,,,,
2002-02-11,1111.939941,144.369995,341.942627,250.520004,138.369995,241.300003,341.929993,378.609985,221.160004,133.800003,92.480003,205.320007
2002-02-12,1107.500000,143.529999,337.042725,248.550003,137.410004,240.229996,339.829987,382.269989,222.020004,134.860001,91.610001,204.009995
2002-02-13,1118.510010,143.720001,342.912628,250.550003,138.309998,243.179993,345.059998,383.420013,223.089996,135.649994,91.989998,205.119995
2002-02-14,1116.479980,142.559998,340.202637,249.850006,137.529999,243.679993,345.779999,381.140015,224.089996,135.039993,92.470001,206.330002
2002-02-15,1104.180054,140.990005,329.372894,248.320007,139.070007,240.279999,339.369995,382.040009,225.660004,133.100006,93.330002,206.839996
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-16,4357.859863,226.899994,2646.169922,852.609985,525.679993,1381.119995,637.309998,1558.130005,766.419983,359.149994,289.839996,540.440002
2022-03-17,4411.669922,229.020004,2664.419922,864.049988,535.909973,1406.859985,645.140015,1583.160034,770.979980,360.790009,293.790009,559.270020
2022-03-18,4463.120117,232.309998,2722.709961,867.469971,539.099976,1437.520020,647.020020,1593.479980,773.039978,357.549988,294.410004,559.780029
